In [3]:
# ================================
# 초보자용: ±2시간 내 활동 유저에서 음성 20명 뽑기 (재현성 완전 보장)
# ================================
import pandas as pd
import random
from collections import defaultdict
from datetime import timedelta

# -------------------------------
# 설정
# -------------------------------
NEG_PER_IMP = 20
RNG_SEED = 42                # ← 시드값 지정
random.seed(RNG_SEED)        # ← 랜덤 시드 고정

# 1) 파일 읽기 (헤더 없음 주의)
BEHAVIOR_COLUMNS = ['ImpressionID', 'UserID', 'Time', 'History', 'Impressions']
beh = pd.read_csv(
    'download/MINDsmall_train/behaviors.tsv',
    sep='\t',
    names=BEHAVIOR_COLUMNS,
    header=None,
    dtype=str
)

# (선택) 뉴스 시간표가 있으면 허용할 뉴스만 제한
news_df = pd.read_csv('news_times_global.csv', parse_dates=['publish_time','lifespan_end'])
if 'NewsID' in news_df.columns:
    news_id_col = 'NewsID'
elif 'news_id' in news_df.columns:
    news_id_col = 'news_id'
else:
    raise ValueError("news_times_global.csv에 NewsID 또는 news_id 컬럼이 필요합니다.")
allowed_news_ids = sorted(set(news_df[news_id_col].astype(str)))  # ← 순서 고정

# 2) 시간 타입으로 바꾸기 (문제 생기면 format= 지정)
beh['Time'] = pd.to_datetime(beh['Time'], errors='coerce')

# 3) 각 줄에서 "클릭된 뉴스(라벨 1)"만 뽑기
user_clicks = defaultdict(list)        # uid -> [news_id, ...]
news_click_users = defaultdict(list)   # news_id -> [(uid, time), ...]

for _, row in beh.iterrows():
    uid = str(row['UserID'])
    t   = row['Time']
    imps = str(row['Impressions']) if pd.notna(row['Impressions']) else ''
    if pd.isna(t) or not imps:
        continue

    for token in imps.split():
        if '-' not in token:
            continue
        nid, label = token.split('-')
        if label == '1':
            user_clicks[uid].append(nid)
            news_click_users[nid].append((uid, t))

# 4) 전체 유저 목록 (후보 만들 때 씀)
all_users = sorted(set(beh['UserID'].astype(str).tolist()))  # ← 순서 고정

# 5) “사용자 활동 기록(누가 언제 활동했나)” 테이블 만들기
active_log = beh[['UserID', 'Time']].dropna().copy()
active_log['UserID'] = active_log['UserID'].astype(str)

# 6) 최종 출력용 리스트
rows = []
impression_id = 1
skipped = 0

# 7) 본격 생성: 각 뉴스의 각 “클릭 이벤트”마다 한 줄 생성
for news_id in allowed_news_ids:  # ← 순서 고정
    clicked_events = news_click_users.get(news_id, [])
    # 클릭 이벤트 순서 고정: 시간 → 유저 순
    clicked_events = sorted(clicked_events, key=lambda x: (x[1], x[0]))

    if not clicked_events:
        continue

    clicked_users_for_this_news = set(uid for uid, _ in clicked_events)

    for pos_uid, pos_time in clicked_events:
        # (a) ±2시간 창
        start_time = pos_time - timedelta(hours=2)
        end_time   = pos_time + timedelta(hours=2)

        # (b) 시간 창 내 활동한 사용자
        mask = (active_log['Time'] >= start_time) & (active_log['Time'] <= end_time)
        active_users_in_window = set(active_log.loc[mask, 'UserID'].tolist())

        # (c) 후보 = 창 내 활동자 - 클릭유저 전체 - 본인
        candidates = active_users_in_window - clicked_users_for_this_news
        if pos_uid in candidates:
            candidates.remove(pos_uid)
        candidates = sorted(list(candidates))  # ← 순서 고정

        # (d) 후보가 20명 미만이면 건너뜀
        if len(candidates) < NEG_PER_IMP:
            skipped += 1
            continue

        # (e) 음성 20명 샘플링 (시드 영향 받음)
        neg_users = random.sample(candidates, NEG_PER_IMP)

        # (f) 양성 + 음성 섞기
        impression_users = [f"{pos_uid}-1"] + [f"{u}-0" for u in neg_users]
        random.shuffle(impression_users)  # 시드 영향 받음

        # (g) 결과 저장
        rows.append({
            'ImpressionID': impression_id,
            'NewsID': news_id,
            'Time': pos_time.strftime('%Y-%m-%d %H:%M:%S'),
            'ImpressionUsers': ' '.join(impression_users)
        })
        impression_id += 1

# 8) 저장
out_df = pd.DataFrame(rows, columns=['ImpressionID','NewsID','Time','ImpressionUsers'])
out_df.to_csv('new_train_user_dataset.tsv', sep='\t', index=False, header=False)

print("생성 행 수:", len(out_df))
print("후보 < 20이라 스킵된 행 수:", skipped)
print(out_df.head(5))


생성 행 수: 236344
후보 < 20이라 스킵된 행 수: 0
   ImpressionID  NewsID                 Time  \
0             1  N10032  2019-11-14 17:38:48   
1             2  N10051  2019-11-14 19:32:44   
2             3  N10056  2019-11-11 21:46:13   
3             4  N10056  2019-11-11 23:21:32   
4             5  N10056  2019-11-12 04:07:09   

                                     ImpressionUsers  
0  U83121-0 U19326-0 U21331-0 U25246-0 U74464-0 U...  
1  U1673-0 U44952-1 U17127-0 U49116-0 U2295-0 U86...  
2  U18416-0 U55331-0 U3560-0 U85676-0 U35558-0 U7...  
3  U54605-0 U6868-0 U75330-0 U91082-0 U47968-0 U7...  
4  U16284-0 U87785-0 U60483-0 U31076-0 U25166-1 U...  


In [4]:
# ================================
# 초보자용 검증 스크립트
# ================================
import pandas as pd
from datetime import timedelta

# --------------------------------
# 0) 경로/기본 설정
# --------------------------------
PATH_BEH = 'download/MINDsmall_train/behaviors.tsv'
PATH_USER_DS = 'new_train_user_dataset.tsv'  # 앞에서 생성한 파일

BEHAVIOR_COLUMNS = ['ImpressionID', 'UserID', 'Time', 'History', 'Impressions']

# --------------------------------
# 1) 데이터 불러오기
# --------------------------------
beh = pd.read_csv(
    PATH_BEH,
    sep='\t',
    names=BEHAVIOR_COLUMNS,
    header=None,
    dtype=str
)
beh['Time'] = pd.to_datetime(beh['Time'], errors='coerce')

userds = pd.read_csv(
    PATH_USER_DS,
    sep='\t',
    names=['ImpressionID','NewsID','Time','ImpressionUsers'],
    header=None,
    dtype=str
)
userds['Time'] = pd.to_datetime(userds['Time'], errors='coerce')

# --------------------------------
# 2) 보조 테이블 만들기
#    - news_clickers: 뉴스ID -> (그 뉴스를 클릭한 유저들의 집합)
#    - active_log: (UserID, Time) 활동 로그 (±2시간 검증용)
# --------------------------------
news_clickers = {}  # dict: news_id -> set(user_ids_who_clicked)
for _, row in beh.iterrows():
    uid = str(row['UserID'])
    imps = str(row['Impressions']) if pd.notna(row['Impressions']) else ''
    if not imps:
        continue
    for token in imps.split():
        # token 예시: "N123-1" or "N456-0"
        if '-' not in token:
            continue
        nid, label = token.split('-')
        if label == '1':
            news_clickers.setdefault(nid, set()).add(uid)

active_log = beh[['UserID','Time']].dropna().copy()
active_log['UserID'] = active_log['UserID'].astype(str)

# --------------------------------
# 3) 검증 함수들
# --------------------------------
def parse_impression_users(s):
    """
    'U1-1 U2-0 U3-0 ...' 형식을 파싱하여
    pos(list 한개), negs(list 여러개), 중복여부를 반환
    """
    if pd.isna(s) or not isinstance(s, str):
        return None, [], False

    tokens = s.strip().split()
    pos = []
    negs = []
    seen = set()
    dup = False

    for tok in tokens:
        if '-' not in tok:
            continue
        uid, lab = tok.split('-', 1)
        uid = uid.strip()
        lab = lab.strip()
        if uid in seen:
            dup = True
        seen.add(uid)
        if lab == '1':
            pos.append(uid)
        else:
            negs.append(uid)
    return pos, negs, dup

def users_active_in_window(center_time, hours=2):
    """center_time ± hours 안에 활동한 사용자 집합"""
    start = center_time - timedelta(hours=hours)
    end   = center_time + timedelta(hours=hours)
    mask = (active_log['Time'] >= start) & (active_log['Time'] <= end)
    return set(active_log.loc[mask, 'UserID'].tolist())

# --------------------------------
# 4) 본격 검증 루프
# --------------------------------
total = len(userds)
bad_format = 0          # 포맷 문제: 양성 1명/음성 20명/중복 등
bad_pos_click = 0       # 양성 유저가 진짜로 그 뉴스를 클릭하지 않은 경우
bad_neg_click = 0       # 음성 유저가 사실 그 뉴스를 클릭한 경우
bad_window = 0          # ±2시간 창 안에 활동하지 않은 유저가 있는 경우
examples = []           # 문제 사례 몇 개 저장

for idx, row in userds.iterrows():
    news_id = str(row['NewsID'])
    t = row['Time']
    pos, negs, dup = parse_impression_users(row['ImpressionUsers'])

    # (A) 포맷 체크
    ok_format = True
    reasonA = []
    if t is pd.NaT:
        ok_format = False
        reasonA.append("Time 파싱 실패(NaT)")
    if pos is None:
        ok_format = False
        reasonA.append("ImpressionUsers 파싱 실패")
    else:
        if len(pos) != 1:
            ok_format = False
            reasonA.append(f"양성 수 != 1 (현재 {len(pos)})")
        if len(negs) != 20:
            ok_format = False
            reasonA.append(f"음성 수 != 20 (현재 {len(negs)})")
        if dup:
            ok_format = False
            reasonA.append("ImpressionUsers에 중복 유저 존재")
        # 양성과 음성의 교집합이 없어야 함
        if set(pos).intersection(negs):
            ok_format = False
            reasonA.append("양성과 음성에 같은 유저가 섞여 있음")
    if not ok_format:
        bad_format += 1
        if len(examples) < 5:
            examples.append(("FORMAT", idx, news_id, reasonA))
        # 포맷이 틀리면 다른 검증은 건너뜀
        continue

    pos_uid = pos[0]

    # (B) 양성: 정말로 그 뉴스를 클릭했는지
    pos_ok = True
    reasonB = []
    clicked_set = news_clickers.get(news_id, set())
    if pos_uid not in clicked_set:
        pos_ok = False
        reasonB.append(f"양성 {pos_uid} 가 behaviors에서 뉴스 {news_id} 클릭 흔적 없음")
    if not pos_ok:
        bad_pos_click += 1
        if len(examples) < 5:
            examples.append(("POS", idx, news_id, reasonB))

    # (C) 음성: 그 뉴스를 클릭하지 않았는지
    neg_ok = True
    reasonC = []
    # clicked_set 재사용
    wrong_negs = set(negs).intersection(clicked_set)
    if wrong_negs:
        neg_ok = False
        reasonC.append(f"음성 중 실제 클릭자 포함: {sorted(list(wrong_negs))[:5]} ...")
    if not neg_ok:
        bad_neg_click += 1
        if len(examples) < 5:
            examples.append(("NEG", idx, news_id, reasonC))

    # (D) ±2시간 활동창 검증 (양성+음성 모두 창 안 활동자여야 함)
    win_ok = True
    reasonD = []
    active_users = users_active_in_window(t, hours=2)
    # 양성 체크
    if pos_uid not in active_users:
        win_ok = False
        reasonD.append(f"양성 {pos_uid} 가 ±2시간 창 내 활동 없음")
    # 음성 체크
    bad_negs_inactive = [u for u in negs if u not in active_users]
    if bad_negs_inactive:
        win_ok = False
        reasonD.append(f"음성 중 창 내 활동 없는 유저 {len(bad_negs_inactive)}명 (예: {bad_negs_inactive[:5]})")
    if not win_ok:
        bad_window += 1
        if len(examples) < 5:
            examples.append(("WINDOW", idx, news_id, reasonD))

# --------------------------------
# 5) 결과 요약 출력
# --------------------------------
print("===================================")
print("검증 요약")
print("===================================")
print(f"총 행 수: {total}")
print(f"(A) 포맷 위반: {bad_format}")
print(f"(B) 양성 클릭 불일치: {bad_pos_click}")
print(f"(C) 음성 클릭 불일치: {bad_neg_click}")
print(f"(D) ±2시간 활동창 위반: {bad_window}")

# 문제 사례 일부 출력
if examples:
    print("\n--- 문제 사례(최대 5개) ---")
    for kind, idx, nid, reasons in examples:
        print(f"[{kind}] row_index={idx}, NewsID={nid}")
        for r in reasons:
            print("  -", r)


검증 요약
총 행 수: 236344
(A) 포맷 위반: 0
(B) 양성 클릭 불일치: 0
(C) 음성 클릭 불일치: 0
(D) ±2시간 활동창 위반: 0


In [5]:
# ================================
# dev: ±2시간 내 활동 유저에서 음성 20명 샘플 (재현성 완전 보장)
# ================================
import pandas as pd
import random
from collections import defaultdict
from datetime import timedelta

# -------------------------
# 기본 설정
# -------------------------
NEG_PER_IMP = 20
RNG_SEED = 42           # 재현성 유지용 시드
random.seed(RNG_SEED)   # 시드 고정

BEHAVIOR_COLUMNS = ['ImpressionID', 'UserID', 'Time', 'History', 'Impressions']

# 1) dev behaviors.tsv 읽기 (헤더 없음!)
dev = pd.read_csv(
    'download/MINDsmall_dev/behaviors.tsv',
    sep='\t',
    names=BEHAVIOR_COLUMNS,
    header=None,
    dtype=str
)

# 시간 타입으로 변환 (문제 생기면 format= 지정)
dev['Time'] = pd.to_datetime(dev['Time'], errors='coerce')

# 2) 허용 뉴스 집합 준비 (newsID 컬럼명 안전화)
news_df = pd.read_csv("news_times_global.csv", parse_dates=['publish_time','lifespan_end'])
if 'NewsID' in news_df.columns:
    news_id_col = 'NewsID'
elif 'news_id' in news_df.columns:
    news_id_col = 'news_id'
else:
    raise ValueError("news_times_global.csv에 NewsID/news_id 컬럼이 필요합니다.")
allowed_news_ids = sorted(set(news_df[news_id_col].astype(str)))  # 순서 고정

# 3) dev impression에서 클릭 뉴스 개수 확인 (옵션)
click_news_count = 0
for impression_row in dev["Impressions"]:
    if pd.isna(impression_row):
        continue
    for token in str(impression_row).split():
        if '-' not in token:
            continue
        nid, lab = token.split('-')
        if lab == '1':
            click_news_count += 1
print('dev impressions-1인 뉴스 개수:', click_news_count)

# 4) 유저별 클릭 리스트 만들기 (label==1만)
dev_user_list = []
for _, row in dev.iterrows():
    uid = str(row['UserID'])
    t   = row['Time']
    imps = str(row['Impressions']) if pd.notna(row['Impressions']) else ''
    click_list = []
    if pd.notna(t) and imps:
        for token in imps.split():
            if '-' not in token:
                continue
            nid, lab = token.split('-')
            if lab == '1':
                click_list.append(nid)
    dev_user_list.append({
        'UserID': uid,
        'Time': t,
        'ClickNews': click_list
    })

# 5) 전체 유저 집합 (순서 고정)
all_user_ids = sorted(set(u['UserID'] for u in dev_user_list))

# 6) 뉴스별 -> (클릭한 유저, 클릭시각) 리스트
news_click_users = defaultdict(list)
for u in dev_user_list:
    uid = u['UserID']
    t   = u['Time']
    for nid in u['ClickNews']:
        news_click_users[nid].append((uid, t))

# 7) 활동 로그 테이블 (±2시간 창 필터용)
active_log = dev[['UserID', 'Time']].dropna().copy()
active_log['UserID'] = active_log['UserID'].astype(str)

def users_in_window(center_time, hours=2):
    """center_time ± hours 안에 활동한 사용자 집합"""
    start = center_time - timedelta(hours=hours)
    end   = center_time + timedelta(hours=hours)
    mask = (active_log['Time'] >= start) & (active_log['Time'] <= end)
    return set(active_log.loc[mask, 'UserID'].tolist())

# 8) 사용자-데이터셋 생성
dev_rows = []
impression_id = 1
skipped = 0

for news_id in allowed_news_ids:  # 순서 고정
    clicked_events = news_click_users.get(news_id, [])
    # 클릭 이벤트 순서 고정 (시간 → 유저)
    clicked_events = sorted(clicked_events, key=lambda x: (x[1], x[0]))

    if not clicked_events:
        continue

    # 이 뉴스를 클릭한 전체 유저 (음성 후보에서 제외)
    clicked_users_for_news = set(uid for uid, _ in clicked_events)

    for pos_uid, pos_time in clicked_events:
        # pos_time이 NaT면 건너뜀
        if pd.isna(pos_time):
            skipped += 1
            continue

        # (1) ±2시간 창의 활동 유저 집합
        active_users = users_in_window(pos_time, hours=2)

        # (2) 후보 = 창 내 활동자 - (이 뉴스 클릭자 전체) - (양성 본인)
        candidates = active_users - clicked_users_for_news
        if pos_uid in candidates:
            candidates.remove(pos_uid)
        candidates = sorted(list(candidates))  # 순서 고정

        # (3) 후보가 20명 미만이면 패스
        if len(candidates) < NEG_PER_IMP:
            skipped += 1
            continue

        # (4) 음성 20명 샘플링
        neg_users = random.sample(candidates, NEG_PER_IMP)

        # (5) 양성 + 음성 합치고 섞기
        impression_users = [f"{pos_uid}-1"] + [f"{u}-0" for u in neg_users]
        random.shuffle(impression_users)

        # (6) 기록
        dev_rows.append({
            'ImpressionID': impression_id,
            'NewsID': news_id,
            'Time': pos_time.strftime('%Y-%m-%d %H:%M:%S'),
            'ImpressionUsers': ' '.join(impression_users)
        })
        impression_id += 1

# 9) 저장
out_cols = ['ImpressionID','NewsID','Time','ImpressionUsers']
dev_out_df = pd.DataFrame(dev_rows, columns=out_cols)
dev_out_df.to_csv('new_dev_user_dataset.tsv', sep='\t', index=False, header=False)

print(f"dev 생성 행 수: {len(dev_out_df)}")
print(f"후보 < {NEG_PER_IMP} 이거나 시간 파싱 문제로 스킵된 행 수: {skipped}")
print(dev_out_df.head(3))


dev impressions-1인 뉴스 개수: 111383
dev 생성 행 수: 111383
후보 < 20 이거나 시간 파싱 문제로 스킵된 행 수: 0
   ImpressionID  NewsID                 Time  \
0             1  N10032  2019-11-15 03:49:09   
1             2  N10050  2019-11-15 18:14:23   
2             3  N10051  2019-11-15 00:21:16   

                                     ImpressionUsers  
0  U9936-0 U14088-0 U7539-0 U75745-0 U24686-0 U23...  
1  U84737-1 U19835-0 U18497-0 U19748-0 U88546-0 U...  
2  U47701-0 U47375-0 U41787-0 U47902-0 U19873-0 U...  


In [7]:
# ================================
# dev 사용자-데이터셋 검증 (수정판)
# ================================
import pandas as pd
from datetime import timedelta

# --- 경로 ---
PATH_BEH = 'download/MINDsmall_dev/behaviors.tsv'   # <<< dev 로 변경!
PATH_USER_DS = 'new_dev_user_dataset.tsv'           # 생성된 dev 사용자-데이터셋

BEHAVIOR_COLUMNS = ['ImpressionID', 'UserID', 'Time', 'History', 'Impressions']

# 1) 데이터 불러오기
beh = pd.read_csv(
    PATH_BEH,
    sep='\t',
    names=BEHAVIOR_COLUMNS,
    header=None,
    dtype=str
)
beh['Time'] = pd.to_datetime(beh['Time'], errors='coerce')

userds = pd.read_csv(
    PATH_USER_DS,
    sep='\t',
    names=['ImpressionID','NewsID','Time','ImpressionUsers'],
    header=None,
    dtype=str
)
userds['Time'] = pd.to_datetime(userds['Time'], errors='coerce')

# 2) 보조 테이블
news_clickers = {}  # news_id -> set(clicked user ids)
for _, row in beh.iterrows():
    uid = str(row['UserID'])
    imps = str(row['Impressions']) if pd.notna(row['Impressions']) else ''
    if not imps:
        continue
    for token in imps.split():
        if '-' not in token:
            continue
        nid, label = token.split('-')
        if label == '1':
            news_clickers.setdefault(nid, set()).add(uid)

active_log = beh[['UserID','Time']].dropna().copy()
active_log['UserID'] = active_log['UserID'].astype(str)

def parse_impression_users(s):
    if pd.isna(s) or not isinstance(s, str):
        return None, [], False
    tokens = s.strip().split()
    pos, negs, seen, dup = [], [], set(), False
    for tok in tokens:
        if '-' not in tok:
            continue
        uid, lab = tok.split('-', 1)
        uid = uid.strip(); lab = lab.strip()
        if uid in seen:
            dup = True
        seen.add(uid)
        (pos if lab == '1' else negs).append(uid)
    return pos, negs, dup

def users_active_in_window(center_time, hours=2):
    start = center_time - timedelta(hours=hours)
    end   = center_time + timedelta(hours=hours)
    mask = (active_log['Time'] >= start) & (active_log['Time'] <= end)
    return set(active_log.loc[mask, 'UserID'].tolist())

# 3) 검증
total = len(userds)
bad_format = bad_pos_click = bad_neg_click = bad_window = 0
examples = []

for idx, row in userds.iterrows():
    news_id = str(row['NewsID'])
    t = row['Time']
    pos, negs, dup = parse_impression_users(row['ImpressionUsers'])

    # (A) 포맷 체크
    ok_format = True
    reasonA = []
    if pd.isna(t):  # <<< is pd.NaT 대신 pd.isna
        ok_format = False; reasonA.append("Time 파싱 실패(NaT)")
    if pos is None:
        ok_format = False; reasonA.append("ImpressionUsers 파싱 실패")
    else:
        if len(pos) != 1:
            ok_format = False; reasonA.append(f"양성 수 != 1 (현재 {len(pos)})")
        if len(negs) != 20:
            ok_format = False; reasonA.append(f"음성 수 != 20 (현재 {len(negs)})")
        if dup:
            ok_format = False; reasonA.append("ImpressionUsers에 중복 유저 존재")
        if set(pos).intersection(negs):
            ok_format = False; reasonA.append("양성과 음성에 같은 유저가 섞여 있음")
    if not ok_format:
        bad_format += 1
        if len(examples) < 5: examples.append(("FORMAT", idx, news_id, reasonA))
        continue

    pos_uid = pos[0]

    # (B) 양성: dev behaviors에서 실제 클릭했는지
    clicked_set = news_clickers.get(news_id, set())
    if pos_uid not in clicked_set:
        bad_pos_click += 1
        if len(examples) < 5:
            examples.append(("POS", idx, news_id, [f"양성 {pos_uid} 가 dev behaviors에서 뉴스 {news_id} 클릭 흔적 없음"]))

    # (C) 음성: dev behaviors에서 클릭하지 않았는지
    wrong_negs = set(negs).intersection(clicked_set)
    if wrong_negs:
        bad_neg_click += 1
        if len(examples) < 5:
            examples.append(("NEG", idx, news_id, [f"음성 중 실제 클릭자 포함: {sorted(list(wrong_negs))[:5]} ..."]))

    # (D) ±2시간 활동창: dev behaviors 기준으로 활동했는지
    active_users = users_active_in_window(t, hours=2)
    win_ok = True; reasonD = []
    if pos_uid not in active_users:
        win_ok = False; reasonD.append(f"양성 {pos_uid} 가 ±2시간 창 내 활동 없음")
    bad_negs_inactive = [u for u in negs if u not in active_users]
    if bad_negs_inactive:
        win_ok = False; reasonD.append(f"음성 중 창 내 활동 없는 유저 {len(bad_negs_inactive)}명 (예: {bad_negs_inactive[:5]})")
    if not win_ok:
        bad_window += 1
        if len(examples) < 5: examples.append(("WINDOW", idx, news_id, reasonD))

# 4) 요약
print("===================================")
print("검증 요약")
print("===================================")
print(f"총 행 수: {total}")
print(f"(A) 포맷 위반: {bad_format}")
print(f"(B) 양성 클릭 불일치: {bad_pos_click}")
print(f"(C) 음성 클릭 불일치: {bad_neg_click}")
print(f"(D) ±2시간 활동창 위반: {bad_window}")

if examples:
    print("\n--- 문제 사례(최대 5개) ---")
    for kind, idx, nid, reasons in examples:
        print(f"[{kind}] row_index={idx}, NewsID={nid}")
        for r in reasons:
            print("  -", r)


검증 요약
총 행 수: 111383
(A) 포맷 위반: 0
(B) 양성 클릭 불일치: 0
(C) 음성 클릭 불일치: 0
(D) ±2시간 활동창 위반: 0
